In [1]:
import time
import datetime
from datetime import datetime
import pandas as pd
import numpy as np
from datetime import date, timezone, timedelta

import collections
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import rcParams
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

import pytz
import json
import logging
from pathlib2 import Path
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [2]:
working_folder=str(Path.cwd())
print(str(working_folder))

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

/Users/mattheus/Documents/GitHub/atc-trend


In [3]:
# ### SAVE ATC RAW DATA FOR TESTING ###

# import pyarrow as pa
# import pyarrow.parquet as pq
# # # start = time.time()
# # # table = pa.Table.from_pandas(day_d0_df)
# # # pq.write_table(table, '4.4_data.parquet')
# table = pq.read_table('4.4_data.parquet')
# day_d0_df = table.to_pandas()
# # # end = time.time()
# # # print('Time Elapsed for day_d0_df: '+str(timedelta(seconds=round(end - start,2))))
# len(day_d0_df)

In [8]:
# test = day_d0_df.head(600000)

In [9]:
# test.to_csv('atc_data.csv',index=0)

In [5]:
day_d0_df = pd.read_csv('atc_data.csv', index_col=0)
day_d0_df.head(10)

,itemid,atc,purchased,name,level1_category,level2_category,level3_category,CR
rank,,,,,,,,
1,4316939143,16346,4209,PerySmith Handheld Vacuum Cleaner Xtreme Serie...,Home Appliances,Vacuum,Vacuum Cleaner,0.257494
2,3258100083,9185,4697,Ribena Sparkling Can - Regular (325ml),Groceries & Pets,Beverages,Asian Drinks,0.511377
3,161686959,7153,2234,Nestle MILO Activ-Go Chocolate Malt Powder (2kg),Groceries & Pets,Milk & Chocolate Drink,Chocolate & Malt,0.312317
4,1388469686,7050,2283,Nestle Milo Activ-Go Chocolate Malt Powder Sof...,Groceries & Pets,Milk & Chocolate Drink,Chocolate & Malt,0.323830
5,1388469676,5437,1509,Milo Activ-Go Chocolate Malt RTD (4 Packs x 12...,Groceries & Pets,Milk & Chocolate Drink,Chocolate & Malt,0.277543
6,6548722783,5378,1505,[8.8] Nestle MILO Activ-Go Chocolate Malt Powd...,Groceries & Pets,Milk & Chocolate Drink,Chocolate & Malt,0.279844
7,2258205864,4629,732,(Ready Stock with Bigger Size) Homez Chest Dra...,Home & Living,Home Storage & Organization,Storage Cabinets,0.158134
8,5919672942,4389,2586,FINETOO Seamless Ice Silk Panties for Women Un...,Women Clothes,Lingerie & Nightwear,Panties,0.589200
9,1251562805,4303,184,★ English / Chinese ★ Mijia Youpin Minutes Sma...,Cameras & Drones,Car Cameras,Rearview Camera,0.042761


# UPDATING TABLE 1

In [11]:
### CLEAN QUERIED D-0 ATC RAW DATA ###
day_d0_clean_left_1 = day_d0_df
to_remove = ['ready','stock','in malaysia']
pat = r'({})'.format('|'.join(to_remove))
day_d0_clean_left_1['name'] = day_d0_clean_left_1['name'].str.lower().str.replace(pat, '', regex=True).fillna(day_d0_clean_left_1['name'])

### SUM ATC COLUMN ###
# day_d0_clean_left_1 = day_d0_clean_left_1[day_d0_clean_left_1['level1_category'] == 'Home Appliances']
day_d0_clean_left_2 = day_d0_clean_left_1.reindex(day_d0_clean_left_1.index.repeat(day_d0_clean_left_1.atc))

# specify n-grams
c_vec = CountVectorizer(ngram_range=(2, 3))
# input to fit_transform() should be an iterable with strings
ngrams = c_vec.fit_transform(day_d0_clean_left_2['name'].to_list())
# needs to happen after fit_transform()
vocab = c_vec.vocabulary_
terms = c_vec.get_feature_names()
freqs = ngrams.sum(axis=0).A1
result = dict(zip(terms, freqs))

day_d0_final_left = pd.DataFrame(result.items(), columns=['keywords', 'count']).sort_values(by=['count'], ascending = False)
day_d0_final_left['totalwords'] = day_d0_final_left['keywords'].str.split().str.len()
day_d0_final_left['max_totalwords'] = day_d0_final_left.groupby('count')['totalwords'].transform(max)
day_d0_final_left = day_d0_final_left[day_d0_final_left['totalwords'] == day_d0_final_left['max_totalwords']]
day_d0_final_left = day_d0_final_left.rename({'count': 'atc',}, axis=1)
day_d0_final_left = day_d0_final_left[day_d0_final_left.columns.drop(list(day_d0_final_left.filter(regex='totalwords')))]
day_d0_final_left['ranking'] = np.arange(len(day_d0_final_left)) + 1
day_d0_final_left.head(5)

,keywords,atc,ranking
3671343,plus size,113168,1
2881788,long sleeve,110893,2
3773048,pro max,108560,3
4514593,stainless steel,96875,4
2309263,high quality,83212,5


In [12]:
###### SAMPLE DATASET

### SUM PURCHASED COLUMN ###
day_d0_clean_right = day_d0_clean_left_1[day_d0_clean_left_1['purchased'] > 0]
day_d0_clean_right = day_d0_clean_right.reindex(day_d0_clean_right.index.repeat(day_d0_clean_right.purchased))

# specify n-grams
c_vec = CountVectorizer(ngram_range=(2, 3))
# input to fit_transform() should be an iterable with strings
ngrams = c_vec.fit_transform(day_d0_clean_right['name'].to_list())
# needs to happen after fit_transform()
vocab = c_vec.vocabulary_
terms = c_vec.get_feature_names()
freqs = ngrams.sum(axis=0).A1
result = dict(zip(terms, freqs))

day_d0_final_right = pd.DataFrame(result.items(), columns=['keywords', 'count']).sort_values(by=['count'], ascending = False)
day_d0_final_right['totalwords'] = day_d0_final_right['keywords'].str.split().str.len()
day_d0_final_right['max_totalwords'] = day_d0_final_right.groupby('count')['totalwords'].transform(max)
day_d0_final_right = day_d0_final_right[day_d0_final_right['totalwords'] == day_d0_final_right['max_totalwords']]
day_d0_final_right = day_d0_final_right.rename({'count': 'purchased',}, axis=1)
day_d0_final_right = day_d0_final_right[day_d0_final_right.columns.drop(list(day_d0_final_right.filter(regex='totalwords')))]

### COMBINE ATC & PURCHASED ###
day_d0_final = day_d0_final_left.set_index('keywords').join(day_d0_final_right.set_index('keywords'), how='left')
day_d0_final.head(5)

,atc,ranking,purchased
keywords,,,
plus size,113168,1,38231.0
long sleeve,110893,2,33285.0
pro max,108560,3,43337.0
stainless steel,96875,4,36561.0
high quality,83212,5,31588.0


In [8]:
###### ORIGINAL DATASET

### SUM PURCHASED COLUMN ###
day_d0_clean_right = day_d0_clean_left_1[day_d0_clean_left_1['purchased'] > 0]
day_d0_clean_right = day_d0_clean_right.reindex(day_d0_clean_right.index.repeat(day_d0_clean_right.purchased))

# specify n-grams
c_vec = CountVectorizer(ngram_range=(2, 3))
# input to fit_transform() should be an iterable with strings
ngrams = c_vec.fit_transform(day_d0_clean_right['name'].to_list())
# needs to happen after fit_transform()
vocab = c_vec.vocabulary_
terms = c_vec.get_feature_names()
freqs = ngrams.sum(axis=0).A1
result = dict(zip(terms, freqs))

day_d0_final_right = pd.DataFrame(result.items(), columns=['keywords', 'count']).sort_values(by=['count'], ascending = False)
day_d0_final_right['totalwords'] = day_d0_final_right['keywords'].str.split().str.len()
day_d0_final_right['max_totalwords'] = day_d0_final_right.groupby('count')['totalwords'].transform(max)
day_d0_final_right = day_d0_final_right[day_d0_final_right['totalwords'] == day_d0_final_right['max_totalwords']]
day_d0_final_right = day_d0_final_right.rename({'count': 'purchased',}, axis=1)
day_d0_final_right = day_d0_final_right[day_d0_final_right.columns.drop(list(day_d0_final_right.filter(regex='totalwords')))]

### COMBINE ATC & PURCHASED ###
day_d0_final = day_d0_final_left.set_index('keywords').join(day_d0_final_right.set_index('keywords'), how='left')
day_d0_final.head(5)

,atc,ranking,purchased
keywords,,,
long sleeve,131674,1,37153.0
plus size,131405,2,41843.0
pro max,126337,3,48502.0
stainless steel,126259,4,43529.0
phone case,101454,5,39934.0


In [9]:
### FINAL D-0 DATA ###
table1_left = day_d0_final
table1_left = table1_left.reset_index()
table1_left = table1_left.rename(columns={'Keywords':'keywords'})
table1_left['purchased'] = table1_left['purchased'].replace(np.nan,0)
table1_left['purchased'] = table1_left['purchased'].apply(lambda d: int(d))
table1_left['CR'] = round(table1_left['purchased'] / table1_left['atc'],2)
table1_left = table1_left.head(100)
table1_left = table1_left[['ranking','keywords','atc','purchased','CR']]

### MERGE D-0 AND D-1 DATA THEN UPDATE GSHEET ###
table1_right = day_d1_df[['keywords','ranking','atc']]
table1_final = table1_left.merge(table1_right, how='left', on='keywords')

LIST_OF_CLIENTS = refresh_client_emails()
try_set_with_dataframe(key, 'Mock Up - Daily', table1_final, row=14, col=1, header=False)
table1_final

NameError: name 'day_d1_df' is not defined

# UPDATING TABLE 2

In [ ]:
table2_left = day_d1_df
table2_left['purchased'] = table2_left['purchased'].replace(np.nan,0)
table2_left['purchased'] = table2_left['purchased'].apply(lambda d: int(float(d)))
table2_left['atc'] = table2_left['atc'].apply(lambda d: int(d))
table2_left['CR'] = round(table2_left['purchased'] / table2_left['atc'],2)
table2_left = table2_left.head(100)
table2_left = table2_left[['ranking','keywords','atc','purchased','CR']]

table2_left.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ranking    100 non-null    object 
 1   keywords   100 non-null    object 
 2   atc        100 non-null    int64  
 3   purchased  100 non-null    int64  
 4   CR         100 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ KB


In [ ]:
table2_right = day_d2_df[['keywords','ranking','atc']]
table2_final = table2_left.merge(table2_right, how='left', on='keywords')

LIST_OF_CLIENTS = refresh_client_emails()
try_set_with_dataframe(key, 'Mock Up - Daily', table2_final, row=14, col=11, header=False)
table2_final

,ranking_x,keywords,atc_x,purchased,CR,ranking_y,atc_y
0,1,long sleeve,97906,12524,0.13,1,87414
1,2,stainless steel,83991,12208,0.15,3,67996
2,3,plus size,83551,12545,0.15,2,76283
3,4,short sleeve,63103,8737,0.14,4,56998
4,5,pro max,62435,10603,0.17,5,56090
...,...,...,...,...,...,...,...
95,96,korean version,14114,1527,0.11,100,12264
96,97,back cover,14063,2568,0.18,103,12143
97,98,premium quality,14062,2592,0.18,93,12722
98,99,women fashion,14023,1902,0.14,90,12957
